<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Drug Label Preprocessing, OpenFDA

In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)

import os
import pprint
import json
import lxml.etree as etree
import pandas as pd
from pandas.io.json import json_normalize
pprint = pprint.pprint #Set up pretty print

In [2]:
data_json_list = []
for i in range(1,8):
    data_json_list.append(json.load(open(f'../test_data/drug-label-000{i}-of-0007.json')))

In [3]:
len(data_json_list)

7

In [4]:
#df = pd.DataFrame(data['results'])
df_list = []
for json in data_json_list:
    df_list.append(json_normalize(json['results']))


In [21]:
df_all = pd.concat(df_list, ignore_index=True)

/home/bpollack/conda_envs/drugs/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [22]:
column_list = []
for df in df_list:
    print(df.info())
    column_list.append(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 152 entries, abuse to when_using_table
dtypes: object(152)
memory usage: 23.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 157 entries, abuse to when_using_table
dtypes: object(157)
memory usage: 24.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 159 entries, abuse to when_using_table
dtypes: object(159)
memory usage: 24.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 152 entries, abuse to when_using
dtypes: object(152)
memory usage: 23.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 163 entries, abuse to when_using_table
dtypes: object(163)
memory usage: 24.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 155 entries, abuse to when_using_table
dtypes: object(155)
me

In [24]:
# see what columns are not in some files, looks mostly like specialty stuff and tables
set(column_list[0].values).difference(set(column_list[1].values))

{'ask_doctor_or_pharmacist_table',
 'ask_doctor_table',
 'assembly_or_installation_instructions',
 'calibration_instructions',
 'cleaning',
 'do_not_use_table',
 'safe_handling_warning_table',
 'statement_of_identity_table',
 'troubleshooting'}

I think we might be able to work with this now.  We've used `json_normalize` to flatten the semi-structured format into something tabular.  There are 153 column labels, but we'll want to drop a bunch and just use a subset for now. I want to just look at the 'openfda' subset of columns, as it looks like there's some useful info there (https://open.fda.gov/apis/openfda-fields/).

In [25]:
df_all_trim = df_all[df_all.columns.drop(list(df_all.filter(regex='table')))]
df_all_trim.head()

,abuse,accessories,active_ingredient,adverse_reactions,alarms,animal_pharmacology_and_or_toxicology,ask_doctor,ask_doctor_or_pharmacist,assembly_or_installation_instructions,boxed_warning,...,summary_of_safety_and_effectiveness,teratogenic_effects,troubleshooting,use_in_specific_populations,user_safety_warnings,version,veterinary_indications,warnings,warnings_and_cautions,when_using
0,NaN,NaN,[HPUS Active Ingredients Each dose contains eq...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,"[Warnings If symptoms persist or worsen, consu...",NaN,NaN
1,NaN,NaN,NaN,[6 ADVERSE REACTIONS Use of levocetirizine dih...,NaN,[13.2 Animal Toxicology Reproductive Toxicolog...,NaN,NaN,NaN,NaN,...,NaN,"[Teratogenic Effects: In rats and rabbits, lev...",NaN,[8 USE IN SPECIFIC POPULATIONS 1.Renal Impairm...,NaN,1,NaN,NaN,[5 WARNINGS AND PRECAUTIONS 1.Avoid engaging i...,NaN
2,NaN,NaN,[Active ingredient (in each extended-release t...,NaN,NaN,NaN,[Ask a doctor before use if you have •persiste...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,[Warnings Do not use •for children under 12 ye...,NaN,NaN
3,NaN,NaN,[ACTIVE INGREDIENT Active Ingredient: Hamameli...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,[WARNINGS Warnings: 1. If the following sympto...,NaN,NaN
4,NaN,NaN,[Active ingredient Benzoyl Peroxide 2.5%],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,[Warnings For external use only],NaN,[When Using This Product • avoid unnecessary s...


Ok, looking good.  Before we play with the data, we have to clean it up.  Are these brackets strings that were parsed wrong or actual list objects?

That answers that.  I don't want a bunch of lists for anything that's a single item, so lets unpack all lists of length=1.  Lets also strip 'openfda' off of our columns.

In [26]:
df_all_trim = df_all_trim.applymap(lambda x: x[0] if type(x) is list and len(x)==1 else x)

In [27]:
df_all_trim.head()

,abuse,accessories,active_ingredient,adverse_reactions,alarms,animal_pharmacology_and_or_toxicology,ask_doctor,ask_doctor_or_pharmacist,assembly_or_installation_instructions,boxed_warning,...,summary_of_safety_and_effectiveness,teratogenic_effects,troubleshooting,use_in_specific_populations,user_safety_warnings,version,veterinary_indications,warnings,warnings_and_cautions,when_using
0,NaN,NaN,HPUS Active Ingredients Each dose contains equ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,"Warnings If symptoms persist or worsen, consul...",NaN,NaN
1,NaN,NaN,NaN,6 ADVERSE REACTIONS Use of levocetirizine dihy...,NaN,13.2 Animal Toxicology Reproductive Toxicology...,NaN,NaN,NaN,NaN,...,NaN,"Teratogenic Effects: In rats and rabbits, levo...",NaN,8 USE IN SPECIFIC POPULATIONS 1.Renal Impairme...,NaN,1,NaN,NaN,5 WARNINGS AND PRECAUTIONS 1.Avoid engaging in...,NaN
2,NaN,NaN,Active ingredient (in each extended-release ta...,NaN,NaN,NaN,Ask a doctor before use if you have •persisten...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,Warnings Do not use •for children under 12 yea...,NaN,NaN
3,NaN,NaN,ACTIVE INGREDIENT Active Ingredient: Hamamelis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,WARNINGS Warnings: 1. If the following symptom...,NaN,NaN
4,NaN,NaN,Active ingredient Benzoyl Peroxide 2.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,Warnings For external use only,NaN,When Using This Product • avoid unnecessary su...


Only two columns are completely unique and do not contain lists: spl_id and spl_set_id.  Brand Names is not unique, but it's close.  Apparently a lot of companies make 'Oxygen'.  Good, I'm a firm believer that oxygen is important.  Let's take a look.

In [13]:
#%env HDF5_USE_FILE_LOCKING=FALSE
#df_all_trim.to_hdf('../test_data/drug_labels.h5', key='all_labels', mode='w')

env: HDF5_USE_FILE_LOCKING=FALSE


/home/bpollack/conda_envs/drugs/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['abuse', 'accessories', 'active_ingredient', 'adverse_reactions', 'alarms', 'animal_pharmacology_and_or_toxicology', 'ask_doctor', 'ask_doctor_or_pharmacist', 'assembly_or_installation_instructions', 'boxed_warning', 'calibration_instructions', 'carcinogenesis_and_mutagenesis_and_impairment_of_fertility', 'cleaning', 'clinical_pharmacology', 'clinical_studies', 'components', 'contraindications', 'controlled_substance', 'dependence', 'description', 'diagram_of_device', 'disposal_and_waste_handling', 'do_not_use', 'dosage_and_administration', 'dosage_forms_and_strengths', 'drug_abuse_and_dependence', 'drug_and_or_laboratory_test_interactions', 'drug_interactions', 'effective_time', 'environmental_warning', 'food_safety_warning'

OverflowError: value too large to convert to int

In [28]:
df_all_trim.to_json('../test_data/drug_labels_all.json')

In [ ]:
df = pd.read_json('../test_data/drug_labels_all.json')